In [1]:
!pip install opendatasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import opendatasets as od
from sklearn.model_selection import train_test_split

In [3]:
od.download("https://www.kaggle.com/datasets/laavanya/stress-level-detection")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: reevald
Your Kaggle Key: ··········


100%|██████████| 1.03M/1.03M [00:00<00:00, 2.04MB/s]

In [4]:
df_data = pd.read_csv("/content/stress-level-detection/Stress-Lysis.csv")
df_data

,Humidity,Temperature,Step count,Stress Level
0,21.33,90.33,123,1
1,21.41,90.41,93,1
2,27.12,96.12,196,2
3,27.64,96.64,177,2
4,10.87,79.87,87,0
...,...,...,...,...
1996,21.82,90.82,96,1
1997,10.45,79.45,45,0
1998,27.22,96.22,135,2
1999,12.46,81.46,64,0


In [28]:
X = df_data.drop("Stress Level", axis=1)

In [29]:
y = df_data['Stress Level']

In [30]:
X

,Humidity,Temperature,Step count
0,21.33,90.33,123
1,21.41,90.41,93
2,27.12,96.12,196
3,27.64,96.64,177
4,10.87,79.87,87
...,...,...,...
1996,21.82,90.82,96
1997,10.45,79.45,45
1998,27.22,96.22,135
1999,12.46,81.46,64


In [31]:
y

0       1
1       1
2       2
3       2
4       0
       ..
1996    1
1997    0
1998    2
1999    0
2000    1
Name: Stress Level, Length: 2001, dtype: int64

In [32]:
y = pd.get_dummies(y).values
y

array([[0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       ...,
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]], dtype=uint8)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [34]:
print("Banyak data train X adalah :", len(X_train))
print("Banyak data test X adalah :", len(X_test))
print("Banyak data train y adalah :", len(y_train))
print("Banyak data test y adalah :", len(y_test))
print("Total dataset adalah :", len(df_data))

Banyak data train X adalah : 1600
Banyak data test X adalah : 401
Banyak data train y adalah : 1600
Banyak data test y adalah : 401
Total dataset adalah : 2001


In [49]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(3)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [50]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 8)                 32        
                                                                 
 dense_13 (Dense)            (None, 16)                144       
                                                                 
 dense_14 (Dense)            (None, 3)                 51        
                                                                 
Total params: 227
Trainable params: 227
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [52]:
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', restore_best_weights=True, patience=20)

In [53]:
model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    callbacks=[callback_es],
    epochs=100)

Epoch 1/100
50/50 [==============================] - 1s 6ms/step - loss: 4.3165 - accuracy: 0.4794 - val_loss: 0.5987 - val_accuracy: 0.7057
Epoch 2/100
50/50 [==============================] - 0s 3ms/step - loss: 0.5075 - accuracy: 0.7625 - val_loss: 0.4364 - val_accuracy: 0.7731
Epoch 3/100
50/50 [==============================] - 0s 3ms/step - loss: 0.3991 - accuracy: 0.8019 - val_loss: 0.3366 - val_accuracy: 0.8828
Epoch 4/100
50/50 [==============================] - 0s 3ms/step - loss: 0.3458 - accuracy: 0.8413 - val_loss: 0.3199 - val_accuracy: 0.8703
Epoch 5/100
50/50 [==============================] - 0s 4ms/step - loss: 0.3131 - accuracy: 0.8731 - val_loss: 0.2763 - val_accuracy: 0.9052
Epoch 6/100
50/50 [==============================] - 0s 4ms/step - loss: 0.2971 - accuracy: 0.8875 - val_loss: 0.2781 - val_accuracy: 0.9077
Epoch 7/100
50/50 [==============================] - 0s 3ms/step - loss: 0.2805 - accuracy: 0.8994 - val_loss: 0.2660 - val_accuracy: 0.9152
Epoch 8/100
5

In [54]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.10361292213201523
Test accuracy: 1.0


In [65]:
x_sample = X_test.sample(1)
x_sample

,Humidity,Temperature,Step count
31,25.46,94.46,172


In [68]:
x_label_true = df_data.iloc[x_sample.index.values[0]]['Stress Level']
print("Level Stress Sebenarnya :", x_label_true)

Level Stress Sebenarnya : 2.0


In [57]:
df_data['Stress Level'].unique()

array([1, 2, 0])

In [58]:
x_sample.shape

(1, 3)

In [59]:
x_sample.values

array([[ 21.5,  90.5, 102. ]])

In [66]:
prediksi_sample = model.predict(x_sample)

1/1 [==============================] - 0s 19ms/step


In [67]:
index_class = np.argmax(prediksi_sample)
print("Level Stress :")
if index_class == 0:
  print('0')
elif index_class == 1:
  print('1')
else:
  print('2')

Level Stress :
2


In [62]:
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflowjs-4.1.0-py3-none-any.whl (84 kB)
     |████████████████████████████████| 40 kB 5.2 MB/s 
     |████████████████████████████████| 588.3 MB 21 kB/s 
     |████████████████████████████████| 16.2 MB 53.8 MB/s 
  Using cached flax-0.6.3-py3-none-any.whl (197 kB)
     |████████████████████████████████| 154 kB 80.2 MB/s 
     |████████████████████████████████| 66 kB 5.0 MB/s 
     |████████████████████████████████| 8.3 MB 43.7 MB/s 
     |████████████████████████████████| 237 kB 82.2 MB/s 
     |████████████████████████████████| 51 kB 8.0 MB/s 
     |████████████████████████████████| 6.0 MB 51.9 MB/s 
     |████████████████████████████████| 439 kB 78.0 MB/s 
     |████████████████████████████████| 1.7 MB 55.6 MB/s 
     |████████████████████████████████| 85 kB 5.2 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.

In [63]:
model.save("model.h5")

In [64]:
!tensorflowjs_converter --input_format=keras /content/model.h5 /content/dir_model

2022-12-11 14:02:09.144469: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-11 14:02:09.144729: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-11 14:02:09.144763: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
